# Library
****

In [1]:
import pandas as pd
import numpy as np
import copy
import math
from sklearn.model_selection import train_test_split
%matplotlib inline

# Clean & analyze data
****

In [2]:
heart_data = pd.read_csv("/kaggle/input/heart-failure-prediction/heart.csv")


In [3]:
heart_data.shape

(918, 12)

In [4]:
heart_data.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [5]:
heart_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


## Removing object column
****
from info above I will remove object type columns and only use integer and float column

In [7]:
del_col = []
for col in heart_data.columns:
    if heart_data[col].dtype == float or heart_data[col].dtype == int:
        pass
    else:
        del_col.append(col)
    if heart_data[col].isnull().all():
        del_col.append(col)
        
print(del_col)

['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope']


In [8]:
clean_data = heart_data.drop(del_col,axis=1)

In [9]:
clean_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Age           918 non-null    int64  
 1   RestingBP     918 non-null    int64  
 2   Cholesterol   918 non-null    int64  
 3   FastingBS     918 non-null    int64  
 4   MaxHR         918 non-null    int64  
 5   Oldpeak       918 non-null    float64
 6   HeartDisease  918 non-null    int64  
dtypes: float64(1), int64(6)
memory usage: 50.3 KB


## Separate "Heart Disease" Column
****

In [15]:
X_data = clean_data.drop(["HeartDisease"],axis=1).values.tolist()

In [16]:
Y_data = clean_data["HeartDisease"].values.tolist()

In [17]:
print(X_data[:3])
print(Y_data[:3])

[[40.0, 140.0, 289.0, 0.0, 172.0, 0.0], [49.0, 160.0, 180.0, 0.0, 156.0, 1.0], [37.0, 130.0, 283.0, 0.0, 98.0, 0.0]]
[0, 1, 0]


## Feature Normalization
****
In this model I am implementing z-score normalization, adjust your input values as shown in this formula:
$$x^{(i)}_j = \dfrac{x^{(i)}_j - \mu_j}{\sigma_j} \tag{1}$$ 

In [18]:
def zscore_normalize_features(X):
    mu = np.mean(X, axis=0)
    sigma = np.std(X, axis=0)
    X_norm = (X - mu)/sigma
    return (X_norm,sigma,mu)

In [20]:
X_norm, X_mu, X_sigma = zscore_normalize_features(X_data)
print(f"X_mu = {X_mu}, \nX_sigma = {X_sigma}")
print(f"Peak to Peak range by column in Raw        X:{np.ptp(X_data,axis=0)}")   
print(f"Peak to Peak range by column in Normalized X:{np.ptp(X_norm,axis=0)}")

X_mu = [  9.42747752  18.50406741 109.3245509    0.42281514  25.44646308
   1.06598907], 
X_sigma = [ 53.51089325 132.39651416 198.79956427   0.23311547 136.80936819
   0.88736383]
Peak to Peak range by column in Raw        X:[ 49.  200.  603.    1.  142.    8.8]
Peak to Peak range by column in Normalized X:[ 5.19757272 10.80843447  5.51568696  2.36509977  5.58034331  8.2552441 ]


In [21]:
print(X_norm.shape)
print(X_norm[:2])

(918, 6)
[[-1.4331398   0.41090889  0.82507026 -0.55134134  1.38292822 -0.83243239]
 [-0.47848359  1.49175234 -0.17196105 -0.55134134  0.75415714  0.10566353]]


## Splitting data into train and test
****
in this notebook I split the data with ratio 2/3 for training data and 1/3 for test data.

In [22]:
X_train, X_test, Y_train, Y_test = train_test_split(
    X_norm, Y_data, test_size=0.33, random_state=4
)

## Sigmoid function for logistic regression
****

$$g(z) = \frac{1}{1+e^{-z}} \tag{1} $$ 

In [23]:
def sigmoid(z):
    g = 1/(1 + np.exp(-z))
    return g

## Cost function for logistic regression
****
$$ J(\mathbf{w},b) = \frac{1}{m}\sum_{i=0}^{m-1} \left[ loss(f_{\mathbf{w},b}(\mathbf{x}^{(i)}), y^{(i)}) \right] \tag{1}$$

where
* m is the number of training examples in the dataset


* $loss(f_{\mathbf{w},b}(\mathbf{x}^{(i)}), y^{(i)})$ is the cost for a single data point, which is - 

    $$loss(f_{\mathbf{w},b}(\mathbf{x}^{(i)}), y^{(i)}) = (-y^{(i)} \log\left(f_{\mathbf{w},b}\left( \mathbf{x}^{(i)} \right) \right) - \left( 1 - y^{(i)}\right) \log \left( 1 - f_{\mathbf{w},b}\left( \mathbf{x}^{(i)} \right) \right) \tag{2}$$
    
    
*  $f_{\mathbf{w},b}(\mathbf{x}^{(i)})$ is the model's prediction, while $y^{(i)}$, which is the actual label

*  $f_{\mathbf{w},b}(\mathbf{x}^{(i)}) = g(\mathbf{w} \cdot \mathbf{x^{(i)}} + b)$ where function $g$ is the sigmoid function.
    * It might be helpful to first calculate an intermediate variable $z_{\mathbf{w},b}(\mathbf{x}^{(i)}) = \mathbf{w} \cdot \mathbf{x^{(i)}} + b = w_0x^{(i)}_0 + ... + w_{n-1}x^{(i)}_{n-1} + b$ where $n$ is the number of features, before calculating $f_{\mathbf{w},b}(\mathbf{x}^{(i)}) = g(z_{\mathbf{w},b}(\mathbf{x}^{(i)}))$

In [24]:
def compute_cost(X, y,w,b,lambda_ =1):
    m, n = X.shape
    total_cost = 0
    for i in range(m):
        z = np.dot(X[i],w) + b
        f_x = sigmoid(z)
        total_cost += (-y[i]*np.log(f_x)) - (1 - y[i])*np.log(1-f_x)
    
    total_cost = total_cost/m
    
    return total_cost

## Gradient Descent for logistic regression
****
$$\begin{align*}& \text{repeat until convergence:} \; \lbrace \newline \; & b := b -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial b} \newline       \; & w_j := w_j -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial w_j} \tag{1}  \; & \text{for j := 0..n-1}\newline & \rbrace\end{align*}$$

where, parameters $b$, $w_j$ are all updated simultaniously
b and w equation can be explained by below equation:
$$
\frac{\partial J(\mathbf{w},b)}{\partial b}  = \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - \mathbf{y}^{(i)}) \tag{2}
$$
$$
\frac{\partial J(\mathbf{w},b)}{\partial w_j}  = \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - \mathbf{y}^{(i)})x_{j}^{(i)} \tag{3}
$$

In [25]:
def compute_gradient(X, y, w, b, lambda_ = None):
    m, n = X.shape
    dj_dw = np.zeros(w.shape)
    dj_db = 0.
    
    for i in range(m):
        z_wb = 0
        for j in range(n):
            z_wb += X[i,j]*w[j]
        z_wb += b
        f_wb = sigmoid(z_wb)
        
        dj_db_i = f_wb - y[i]
        dj_db += dj_db_i
        
        for j in range(n):
            dj_dw_ij = dj_db_i*X[i][j]
            dj_dw[j] += dj_dw_ij
    
    dj_dw = dj_dw/m
    dj_db = dj_db/m
    
    return dj_db, dj_dw

In [26]:
def gradient_descent(X, y, w_in, b_in, cost_function, gradient_function, alpha, num_iters, lambda_):
    m = len(X)
    
    J_history = []
    w_history = []
    
    for i in range(num_iters):
        
        dj_db, dj_dw = gradient_function(X, y, w_in, b_in, lambda_)
        
        w_in = w_in - alpha*dj_dw
        b_in = b_in - alpha*dj_db
        
        if(i < 100000):
            cost = cost_function(X, y, w_in, b_in, lambda_)
            J_history.append(cost)
        
        if(i % math.ceil(num_iters/10) == 0 or i == (num_iters-1)):
            w_history.append(w_in)
            print(f"Iteration {i:4}: Cost {float(J_history[-1]):8.2f}   ")
    
    return w_in, b_in, J_history, w_history

## Predict Function
****

In [27]:
def predict(X, w, b):
    m,n = X.shape
    p = np.zeros(m)
    
    for i in range(m):
        z_wb = 0
        for j in range(n):
            z_wb += X[i,j] * w[j]
        z_wb += b
        f_wb = sigmoid(z_wb)
        p[i] = f_wb >= 0.5
    
    return p

## Learning Parameters
****

In [48]:
np.random.seed(1)
initial_w = np.random.rand(X_train.shape[1]) - 0.5
initial_b = 1.
lambda_ = 0.01;
iterations = 10000
alpha = 0.01
print(X_train[:5],initial_w)
w, b, J_history,_ = gradient_descent(X_train, Y_train,initial_w, initial_b, 
                                   compute_cost, compute_gradient, 
                                   alpha, iterations, lambda_)

[[-0.16026485  0.30282455  0.22136323 -0.55134134  1.26503364 -0.83243239]
 [-1.96350436 -0.56185021 -0.06219613 -0.55134134  1.4615246  -0.83243239]
 [-0.26633776 -0.12951283 -0.18110812 -0.55134134 -1.44654163 -0.83243239]
 [ 0.2640268   2.03217406 -1.81843477 -0.55134134 -0.54268321  1.51280741]
 [-0.90277524 -1.21035628  0.59639336 -0.55134134 -0.18899948  0.29328271]] [-0.082978    0.22032449 -0.49988563 -0.19766743 -0.35324411 -0.40766141]
Iteration    0: Cost     0.77   
Iteration 1000: Cost     0.49   
Iteration 2000: Cost     0.49   
Iteration 3000: Cost     0.49   
Iteration 4000: Cost     0.49   
Iteration 5000: Cost     0.49   
Iteration 6000: Cost     0.49   
Iteration 7000: Cost     0.49   
Iteration 8000: Cost     0.49   
Iteration 9000: Cost     0.49   
Iteration 9999: Cost     0.49   


## Evaluation Normal Logistic Regression
****

In [49]:
X_test
p = predict(X_test, w, b)
print('Train Accuracy: %f'%(np.mean(p == Y_test) * 100))

Train Accuracy: 75.907591


In [38]:
#normal logistic regression prediction
Y_test
normal_logistic = pd.DataFrame(Y_test, columns=['Heart Disease Test'])
normal_logistic["prediction"] = p
normal_logistic
normal_logistic.to_csv("normal_logistic.csv", index = None)

# Regularized Logistic Regression
****
In this notebook I will use PolynomialFeatures from ****Sklearn**** for feature mapping, this step is necessary so we can fit our data better.

In [53]:
from sklearn.preprocessing import PolynomialFeatures

In [73]:
poly = PolynomialFeatures(degree=2)
#PolynomialFeatures(interaction_only=True) degree=2

X_mapped = poly.fit_transform(X_train)

In [76]:
print(X_train[:1])
print(X_mapped[:1])

[[-0.16026485  0.30282455  0.22136323 -0.55134134  1.26503364 -0.83243239]]
[[ 1.         -0.16026485  0.30282455  0.22136323 -0.55134134  1.26503364
  -0.83243239  0.02568482 -0.04853213 -0.03547674  0.08836064 -0.20274042
   0.13340965  0.09170271  0.06703422 -0.16695969  0.38308324 -0.25208096
   0.04900168 -0.1220467   0.28003193 -0.18426992  0.30397727 -0.69746534
   0.45895439  1.60031011 -1.05305498  0.69294369]]


## Compute cost for Regularized Logistic Regression
****
$$J(\mathbf{w},b) = \frac{1}{m}  \sum_{i=0}^{m-1} \left[ -y^{(i)} \log\left(f_{\mathbf{w},b}\left( \mathbf{x}^{(i)} \right) \right) - \left( 1 - y^{(i)}\right) \log \left( 1 - f_{\mathbf{w},b}\left( \mathbf{x}^{(i)} \right) \right) \right] + \frac{\lambda}{2m}  \sum_{j=0}^{n-1} w_j^2 \tag{1}$$

As you can see from equation above there is difference in from normal compute cost, that is:
$$\frac{\lambda}{2m}  \sum_{j=0}^{n-1} w_j^2 \tag{2}$$

In [74]:
def compute_cost_reg(X, y, w, b, lambda_ = 1):
    m,n = X.shape
    
    cost_without_reg = compute_cost(X,y,w,b)
    reg_cost = 0.
    #print(m,n)
    #print("cost_without_reg",cost_without_reg)
    for i in range(n):
        reg_cost += w[i]**2
        
    total_cost = cost_without_reg + (lambda_/(2 * m)) * reg_cost
    
    return total_cost

## Gradient Descent for Regularized Logistic Regression
****
$$\frac{\partial J(\mathbf{w},b)}{\partial b} = \frac{1}{m}  \sum_{i=0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)}) \tag{1} $$

$$\frac{\partial J(\mathbf{w},b)}{\partial w_j} = \left( \frac{1}{m}  \sum_{i=0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)}) x_j^{(i)} \right) + \frac{\lambda}{m} w_j  \quad\, \mbox{for $j=0...(n-1)$} \tag{2}$$


As you can see,$\frac{\partial J(\mathbf{w},b)}{\partial b}$ is the same, the difference is the following term in $\frac{\partial J(\mathbf{w},b)}{\partial w}$, which is $$\frac{\lambda}{m} w_j  \quad\, \mbox{for $j=0...(n-1)$} \tag{3} $$ 

So we're going to calculate gradient descent plus equation (3) as code below

In [40]:
def compute_gradient_reg(X, y, w, b, lambda_ = 1):
    m,n = X.shape
    
    dj_db, dj_dw = compute_gradient(X, y, w, b)
    
    for j in range(n):
        dj_dw_j_reg = (lambda_/m)*w[j]
        dj_dw[j] += dj_dw_j_reg
    
    return dj_db, dj_dw

## Evaluation Regularized Logistic Regression
****

In [75]:
np.random.seed(1)
initial_w = np.random.rand(X_mapped.shape[1]) - 0.5
initial_b = 1.
lambda_ = 0.01;
iterations = 10000
alpha = 0.01
w_reg, b_reg, J_history,_ = gradient_descent(X_mapped, Y_train,initial_w, initial_b, 
                                   compute_cost_reg, compute_gradient_reg, 
                                   alpha, iterations, lambda_)

Iteration    0: Cost     1.09   
Iteration 1000: Cost     0.47   
Iteration 2000: Cost     0.46   
Iteration 3000: Cost     0.46   
Iteration 4000: Cost     0.46   
Iteration 5000: Cost     0.46   
Iteration 6000: Cost     0.46   
Iteration 7000: Cost     0.46   
Iteration 8000: Cost     0.46   
Iteration 9000: Cost     0.46   
Iteration 9999: Cost     0.46   


Result for PolynomialFeatures(interaction_only=True)

In [58]:
#PolynomialFeatures(interaction_only=True)
X_test_mapped = poly.fit_transform(X_test)
p = predict(X_test_mapped, w_reg, b_reg)
print('Test Accuracy: %f'%(np.mean(p == Y_test) * 100))

Test Accuracy: 77.887789


Result for PolynomialFeatures(degree=2)

In [77]:
X_test_mapped = poly.fit_transform(X_test)
p = predict(X_test_mapped, w_reg, b_reg)
print('Test Accuracy: %f'%(np.mean(p == Y_test) * 100))

Test Accuracy: 80.198020


In [78]:
#Regularized logistic regression prediction
#Y_test
regularized_logistic = pd.DataFrame(Y_test, columns=['Heart Disease Test'])
regularized_logistic["prediction"] = p
regularized_logistic
regularized_logistic.to_csv("regularized_logistic.csv", index = None)

# SkLearn Logistic Regression
****


In [69]:
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression(C=1000.0, random_state=0)
lr_model.fit(X_train, Y_train)

LogisticRegression(C=1000.0, random_state=0)

In [70]:
y_prediction = lr_model.predict(X_test)
print("Prediction on test set:", y_prediction)

Prediction on test set: [1 1 1 0 0 1 1 0 1 0 0 1 0 0 0 1 0 0 0 1 1 0 0 0 1 1 0 0 1 1 1 1 1 1 0 0 1
 1 1 1 1 0 0 1 1 1 1 1 0 0 0 1 0 0 0 0 0 1 0 1 1 0 0 0 1 1 0 1 0 0 1 1 0 0
 0 1 0 1 0 1 0 1 0 1 1 0 0 0 1 1 1 0 0 0 0 1 0 1 1 1 0 0 1 1 1 1 0 0 1 1 1
 0 1 0 0 0 1 1 0 1 0 1 1 0 1 0 1 0 0 1 1 1 1 1 1 0 1 0 1 1 0 1 0 0 1 1 1 0
 1 1 1 0 0 1 1 0 0 1 1 1 1 1 1 0 1 1 1 1 0 0 1 0 1 1 1 1 0 1 1 1 0 0 0 1 1
 1 0 1 0 1 1 1 1 0 0 1 1 1 0 0 1 0 1 1 1 0 1 0 0 0 1 0 1 1 1 1 1 1 1 1 0 1
 0 1 1 0 1 1 1 0 0 0 1 0 1 0 0 1 1 0 0 1 1 1 0 0 0 0 1 0 1 1 0 1 0 1 1 0 1
 1 1 1 0 1 0 1 1 1 1 0 0 1 0 1 1 0 1 1 1 0 0 1 0 0 1 1 0 0 0 1 0 0 0 1 1 0
 1 0 1 1 0 1 0]


In [71]:
print("Accuracy on training set:", lr_model.score(X_test, Y_test))

Accuracy on training set: 0.759075907590759


In [72]:
print('Test Accuracy: %f'%(np.mean(y_prediction == Y_test) * 100))

Test Accuracy: 75.907591
